<a href="https://colab.research.google.com/github/Kaliostrogoblin/PDD/blob/master/examples/Train_Siamese_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Siamese Network for PDD data

In this example we will show, how to train your own classifier using [Plant Disease Database](http://pdd.jinr.ru/db/)

### Cloning the repo

At first we will clone the repository.

In [1]:
!rm -r -f PDD
!git clone https://github.com/Kaliostrogoblin/PDD.git

Cloning into 'PDD'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 227 (delta 0), reused 2 (delta 0), pack-reused 222
Receiving objects: 100% (227/227), 53.90 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (124/124), done.


Change the directory to PDD to get access of helper functions and classes.

In [2]:
import os
os.chdir('PDD')
# verify if we are in correct directory
os.listdir()

['.gitignore', 'server', 'examples', 'pdd', 'README.md', '.git']

### Downloading the grape pdd dataset 

We load it and split on train and test subdirectories in place.

In [3]:
from pdd.datasets.grape import load_data

train_data_path, test_data_path = load_data(split_on_train_test=True, random_state=13)

Using TensorFlow backend.


34845it [00:10, 3316.43it/s]                             
100%|████████████████████| 10/10 [00:00<00:00, 75.92it/s]

Splitting on train and test...


The structure of the dataset catalogue now:

```
grape
│    
└───train
│   └───black_rot
│   |   │   20.jpg
│   |   │   ...
|   |
|   └───chlorosis
|   └───esca
|   └───healty
|   └───mildew
|
└───test
    └───black_rot
    |   │   3.jpg
    |   │   ...
    |
    └───chlorosis
    └───esca
    └───healty
    └───mildew
```

### Create a feature extractor `twin` and a siamese network

In [0]:
from pdd.models import get_feature_extractor
from pdd.models import make_siamese
import tensorflow.keras.backend as K
import tensorflow as tf

# set the single session for tensorflow and keras both
sess = tf.Session()
K.set_session(sess)

We are using the cross-entropy loss as in [this paper](http://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf) instead of contrastive loss. 

But feel free for using it by changing parameter `loss` to 'contrastive'. E.g.

```python
siams = make_siamese(feature_extractor, dist='l2', loss='contrastive')
```

There are three types of distances:


*   `'l1'`
*   `'l2'`
*   `'cosine'`

**But only `'l1'` is available for cross-entropy loss.**



In [5]:
input_shape = (256, 256, 3)

print("Building feature extractor...")
feature_extractor = get_feature_extractor(input_shape)

print("Constructing siamese network...")
siams = make_siamese(feature_extractor, dist='l1', loss='cross_entropy')
siams.summary()

Building feature extractor...
Instructions for updating:
Colocations handled automatically by placer.
Constructing siamese network...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         14902496    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________

### Creating a batch generator

To train a siamese network data should be passed to the input as **positive-negative pairs**. Positive pair of images means a pair consists of images from the same class. Negative pairs consist of images of different classes.

In [0]:
from pdd.utils.training import SiameseBatchGenerator

For training we are using a strong augmentation including rotations, zooming, flips and channel shifts.

In [0]:
train_batch_gen = SiameseBatchGenerator.from_directory(dirname=train_data_path, augment=True)
test_batch_gen = SiameseBatchGenerator.from_directory(dirname=test_data_path)

def siams_generator(batch_gen, batch_size=None):
    while True:
        batch_xs, batch_ys = batch_gen.next_batch(batch_size)
        yield [batch_xs[0], batch_xs[1]], batch_ys

### Training

Let's train the model. 

**Note**, despite the fact that we are training the ыiamese model, the feature extractor is also being trained.

In [0]:
siams.fit_generator(
    generator=siams_generator(train_batch_gen),
    steps_per_epoch=100,
    epochs=50,
    verbose=1,
    validation_data=siams_generator(test_batch_gen),
    validation_steps=30,
    shuffle=True
)

Epoch 1/50
100/100 [==============================] - 165s 2s/step - loss: 0.7295 - acc: 0.5353 - val_loss: 0.6761 - val_acc: 0.5656
Epoch 2/50
100/100 [==============================] - 158s 2s/step - loss: 0.7211 - acc: 0.5572 - val_loss: 0.6701 - val_acc: 0.5979
Epoch 3/50
100/100 [==============================] - 157s 2s/step - loss: 0.7106 - acc: 0.5672 - val_loss: 0.7013 - val_acc: 0.5781
Epoch 4/50
100/100 [==============================] - 158s 2s/step - loss: 0.7099 - acc: 0.5634 - val_loss: 0.6320 - val_acc: 0.6385
Epoch 5/50
100/100 [==============================] - 157s 2s/step - loss: 0.6636 - acc: 0.6097 - val_loss: 0.6729 - val_acc: 0.6188
Epoch 6/50
100/100 [==============================] - 158s 2s/step - loss: 0.6703 - acc: 0.6059 - val_loss: 0.5866 - val_acc: 0.6854
Epoch 7/50
100/100 [==============================] - 157s 2s/step - loss: 0.6380 - acc: 0.6369 - val_loss: 0.6394 - val_acc: 0.6646
Epoch 8/50
100/100 [==============================] - 158s 2s/step - 

### K-nearest neighbors for classification

Using of a siamese network for classification requires to perform an iterative n-shot task. But to avoid this restriction we can build a KNN classifier with the help of features extracted using the `twin` network.

To significantly speed up the inference phase of the classifer we are going to utilize a fast k-nearest-neighbour search based on the method used in ["Learning To Remember Rare Events"](https://openreview.net/pdf?id=SJTQLdqlg).

#### Save the feature extractor model and clear session

In [0]:
print("Saving feature extractor...")
feature_extractor.save('pdd_feature_extractor.h5')

Saving feature extractor...


In [0]:
K.clear_session()
tf.reset_default_graph()
del sess

#### Load the feature exctrator to KNN model

In [0]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model

from pdd.models import TfKNN
from pdd.utils.data_utils import create_dataset_from_dir

In [14]:
import tensorflow as tf
sess = tf.Session()

from tensorflow.keras import backend as K
K.set_session(sess)

print("Loading feature extractor...")
feature_extractor = load_model("pdd_feature_extractor.h5")

Loading feature extractor...


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


#### Load datasets for the evaluation

In [15]:
print("Loading datasets...")
train_dataset = create_dataset_from_dir(train_data_path, shuffle=True)
test_dataset = create_dataset_from_dir(test_data_path, shuffle=True)

Loading datasets...


#### Create KNN graph

In [0]:
tfknn = TfKNN(sess, 
              feature_extractor, 
              (train_dataset['data'], train_dataset['target']))

Getting keys from support set...
Took 1.48 seconds to run.

Freezing feature extractor graph...
INFO:tensorflow:Froze 52 variables.
INFO:tensorflow:Converted 52 variables to const ops.
Took 0.39 seconds to run.

Creating TfKNN graph...
Took 0.02 seconds to run.



#### Evaluate

In [0]:
# predictions and similarities
preds, sims = tfknn.predict(test_dataset['data'])
accuracy = accuracy_score(test_dataset['target'], preds)
print("Accuracy: %.2f" % accuracy)

Accuracy: 0.93


#### Save the model for tensorflow serving

In [0]:
tfknn.save_graph_for_serving("tfknn_graph")

Saving graph for serving...
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: tfknn_graph/saved_model.pb
Took 0.29 seconds to run.



## Save and upload model to Google Drive

Mount your Google Drive

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Copy files to gdrive

In [0]:
# make directory in the destination
!mkdir "/gdrive/My Drive/pdd_model"

# copy models 
!cp tfknn_graph/saved_model.pb "/gdrive/My Drive/pdd_model/tf_graph.pb"
!cp pdd_feature_extractor.h5 "/gdrive/My Drive/pdd_model/pdd_feature_extractor.h5"